In [3]:
################################################################################                                                                                                                                                              
################################################################################
#
# FILE: crexi-rent-process.ipynb
#
# BY: Dmitry Sedov 
#
# CREATED: Fri Mat 27 2020
#
# DESC: This code processes the rent data downloaded from crexi to put it into a 
#       single dataset.
#
# EXEC:
#      
################################################################################
################################################################################

In [4]:
############################### Libraries ######################################

import pandas as pd
import json
import os

################################################################################

In [5]:
############################### Constants ######################################

output_folder_path = '/home/user/projects/urban/data/input/Rent/Crexi/'

################################################################################

In [8]:
files = [f for f in os.listdir(output_folder_path) if os.path.isfile(os.path.join(output_folder_path, f))]

In [18]:
datasets = []
for f in files:
    with open(os.path.join(output_folder_path, f), 'r') as the_file:
        test = json.load(the_file)
        data = pd.DataFrame(test['Data'])
        data['zip_code'] = f[3:8]
        datasets.append(data)

In [19]:
data = pd.concat(datasets)

In [20]:
subdata = data[data['RateMonthly'].notnull()]

In [22]:
subdata['zip_code'].nunique()

6607